## Usage
- Edit the settings in the cell below.
- Cell -> Run All.
- A Napari window will open, where you can scroll through your data.

In [ ]:

# change this to point to your plate as seen in lmu_active1/instruments/Nano
plate = 'lyu/140324-A53T P62 staining/140324-A53T P62 staining/2024-03-27/20378/TimePoint_1'
plate = 'karkkael/microwell images/Plate1/2024-09-23/1/TimePoint_1/'

# set lmu_active1 root folder for Linux or Windows
#lmu_active1 = Path('/mnt/lmu_active1') # Linux
#lmu_active1 = Path('L:\lmu_active1') # Windows

# define colors you want to use (as many as you have channels)
#colormap = ["yellow", "magenta", "cyan"]
colormap = ["blue", "green", "red"]
colormap = ["blue", "green", "red", "magenta"]


## Code
You don't need to make changes in the code cells below. 

In [ ]:
from aicsimageio.aics_image import AICSImage
from pathlib import Path
import matplotlib.pyplot as plt
import napari
import numpy as np
import os
import pandas as pd
import platform

def get_lmu_active1():
    current_os = platform.system()
    
    if current_os == "Windows":
        return "L:\\lmu_active1"
    elif current_os == "Linux":
        return "/mnt/lmu_active1"
    else:
        raise ValueError(f"Unsupported operating system: {current_os}")
        
# original image folder
orig = get_lmu_active1() / Path('instruments/Nano') / Path(plate)
orig = Path('/home/user/nanodata') / Path(plate)
#orig = Path('/home/hajaalin/data/micro') / Path(plate)


In [ ]:
PATH = 'Path'
DATE = 'Date'
TIMEPOINT = 'TimePoint'
ZSTEP = 'ZStep'
PLATE = 'Plate'
WELL = 'Well'
SITE = 'Site'
CHANNEL = 'Channel'

files = [(str(x),x.parent,x.name) for x in orig.glob("**/*.tif") if not "thumb" in x.name]
df = pd.DataFrame(files, columns=[PATH,'DirName','FileName'])

files = [(str(x)) for x in orig.glob("**/*.tif") if not "thumb" in x.name]
#files = ['/home/user/data/ael/microwell images/Plate1/2024-09-23/1/TimePoint_1/ZStep_9/Plate1_E05_s2_w1D10A7B39-96B7-4967-8EC6-22EA232A7725.tif']
df = pd.DataFrame(files, columns=[PATH])

print(files[-1])
df.head()


metadata_columns = {
    'mc1': DATE,
    'mc2': TIMEPOINT,
    'mc3': ZSTEP,
    'mc4': PLATE,
    'mc5': WELL,
    'mc6': SITE,
    'mc7': CHANNEL
}
# Regular expression pattern
pattern = r'.*/(?P<Date>\d\d\d\d-\d\d-\d\d)/[^/]*/TimePoint_(?P<TimePoint>\d+)/(?:ZStep_(?P<ZStep>\d+)/)?(?P<Plate>[^_]*)_(?P<Well>\w\d{2})_s(?P<Site>\d)_(?P<Wavelength>w\d)'

# Cross-platform pattern with dynamic column names
pattern = r'.*[/\\](?P<{mc1}>\d{{4}}-\d{{2}}-\d{{2}})[/\\][^/\\]*[/\\]TimePoint_(?P<{mc2}>\d+)(?:[/\\]ZStep_(?P<{mc3}>\d+))?[/\\](?P<{mc4}>[^_]+)_(?P<{mc5}>\w\d{{2}})_s(?P<{mc6}>\d)_(?P<{mc7}>w\d)'.format(**metadata_columns)

# Apply the regex pattern and extract the desired columns
df_extracted = df[PATH].str.extract(pattern)
print()

# Add the extracted columns back to the original dataframe
df = df.join(df_extracted)

# Show the result
df.head()

In [ ]:
df.tail()

In [ ]:
files[-1]

In [ ]:
wavelengths = sorted(df.Channel.unique())
wavelengths

In [ ]:
metadata_cols = metadata_columns.values()
metadata_cols

In [ ]:
mask = df[ZSTEP].isnull()
df2d = df[mask].copy().reset_index(drop=True)
df3d = df[~mask].copy().reset_index(drop=True)
df3d[ZSTEP] = df3d[ZSTEP].astype(int)

df2d.sort_values(by=[PLATE, WELL, SITE, CHANNEL], inplace=True, ignore_index=True)
df3d.sort_values(by=[PLATE, WELL, SITE, ZSTEP, CHANNEL], inplace=True, ignore_index=True)
df3d.head()

In [ ]:
grouped2d = df2d.groupby(by=[PLATE, WELL, SITE]).agg(list)
grouped2d

In [ ]:
grouped3d = df3d.groupby(by=[PLATE, WELL, SITE]).agg(list)
grouped3d

In [ ]:
print(len(grouped3d.loc['Plate1', 'C07', '2'][PATH]))
print(len(grouped3d.loc['Plate1', 'C07', '2'][DATE]))
print(len(grouped3d.loc['Plate1', 'C07', '2'][TIMEPOINT]))
print(len(grouped3d.loc['Plate1', 'C07', '2'][ZSTEP]))
print(len(grouped3d.loc['Plate1', 'C07', '2'][CHANNEL]))
print(grouped3d.loc['Plate1', 'C07', '2'])
paths = grouped3d.loc['Plate1', 'C07', '2'][PATH]
for p in paths:
    print(p)

In [ ]:
import dask.array as da
from aicsimageio import AICSImage

# Dictionary to store Dask arrays for each plate
plates = []
plate_stack = None

# Prebuild index mapping
index_map = {}

# Group by plate and well to handle multiple sites within a well
for plate, plate_group in grouped3d.groupby(PLATE):
    wells = []
    
    # Iterate over each well
    for well, well_group in plate_group.groupby(WELL):
        sites = []
        
        # Iterate over each site
        for site, site_group in well_group.groupby(SITE):
            z_steps = []
            
            # At this point, we know the plate, well, and site
            # Add an entry to index_mapping for this site
            index_map[(plate, well, site)] = (len(plates), len(wells), len(sites))

            print(site_group.columns)
            print(site_group.shape)
            print(site_group[ZSTEP].apply(type).unique())  # Check the type of elements in the ZStep column
            #print(site_group[ZSTEP].head())  # Inspect the first few rows
            
            # Explode both ZStep and Channel columns to ensure they correspond correctly
            exploded_df = site_group.explode([PATH, DATE, TIMEPOINT, ZSTEP, CHANNEL])
            print(exploded_df.shape)
            print(exploded_df.apply(type).unique())
            #print(exploded_df.head())

            # Group by ZStep to handle stacking of channels for each Z-slice
            for zstep, zstep_group in exploded_df.groupby(ZSTEP):
                channels = []
                
                # Iterate over each channel and stack them for the current Z-step
                for channel_path in zstep_group[PATH]:
                    print(plate, well, site, zstep, channel_path)
                    img = AICSImage(channel_path)
                    # Use img.get_image_dask_data() for lazy loading of data
                    dask_data = img.get_image_dask_data()
                    #print(dask_data.shape)
                    dask_data = dask_data.squeeze()
                    #print(dask_data.shape)
                    channels.append(dask_data)
    
                print()
                # Stack channels along a new axis (assume channels have same shape)
                z_step_stack = da.stack(channels, axis=0)  # Stack channels for this Z-step
                z_steps.append(z_step_stack)
            
            print()
            # Stack Z-steps into a full 3D array for the current site
            site_stack = da.stack(z_steps, axis=0)  # Stack Z-slices to form a 3D site-level array
            print(site_stack.shape)
            sites.append(site_stack)
        
        # Stack all site-level arrays into a well-level array
        well_stack = da.stack(sites, axis=0)  # Stack sites into a well
        wells.append(well_stack)
    
    # Stack all well-level arrays into a plate-level array
    plate_stack = da.stack(wells, axis=0)  # Stack wells into a plate
    plates.append(plate_stack)
        
final_dask_array = da.stack(plates)
print(final_dask_array.shape)


In [ ]:
print(final_dask_array.shape)
print(index_map)

In [ ]:
final_dask_array[0, 0, 0, :, :, :, :]

In [ ]:
import napari
import dask.array as da
from magicgui import magicgui

# Prebuild the index with (plate, well, site) -> dask slice
#index_map = {}  # assuming this has been built during array construction
plates = list(df3d[PLATE].unique())
wells = list(df3d[WELL].unique())
sites = list(df3d[SITE].unique())

print(plates)
print(wells)
print(sites)

In [ ]:
# Initialize Napari viewer
viewer = napari.Viewer()

# Add Dask array to the viewer with initial data
image_layer = viewer.add_image(final_dask_array[0, 0, 0, :, :, :, :], channel_axis=1)  # Keep channel_axis=1

# Create pull-down for plates and wells
@magicgui(plate={"choices": plates}, well={"choices": wells}, site={"choices": sites, "label": "Site"})
def navigation_widget(plate: str, well: str, site: str):  # Change site type to str
    print(plate, well, site)  # Debug print
    # Select data based on plate, well, and site
    if (plate, well, site) in index_map:
        data_slice = index_map[(plate, well, site)]
        image_layer[0].data = final_dask_array[data_slice]  # Update the image layer
    else:
        print(f"No data found for (plate: {plate}, well: {well}, site: {site})")  # Print message if no data

# Add widget to Napari viewer
viewer.window.add_dock_widget(navigation_widget)
napari.run()


In [ ]:
import napari
from qtpy.QtWidgets import QVBoxLayout, QWidget, QLabel, QComboBox

# Initialize Napari viewer
viewer = napari.Viewer()

# Add Dask array to the viewer with initial data
image_layer = viewer.add_image(final_dask_array[0, 0, 0, :, :, :, :], channel_axis=1)#[0]

# Create a widget for navigation
class NavigationWidget(QWidget):
    def __init__(self, plates, wells, sites):
        super().__init__()
        layout = QVBoxLayout()

        # Plate selection
        self.plate_label = QLabel("Plate")
        self.plate_combo = QComboBox()
        self.plate_combo.addItems(plates)
        self.plate_combo.currentTextChanged.connect(self.update_image)

        # Well selection
        self.well_label = QLabel("Well")
        self.well_combo = QComboBox()
        self.well_combo.addItems(wells)
        self.well_combo.currentTextChanged.connect(self.update_image)

        # Site selection
        self.site_label = QLabel("Site")
        self.site_combo = QComboBox()
        self.site_combo.addItems(sites)
        self.site_combo.currentTextChanged.connect(self.update_image)

        # Adding widgets to layout
        layout.addWidget(self.plate_label)
        layout.addWidget(self.plate_combo)
        layout.addWidget(self.well_label)
        layout.addWidget(self.well_combo)
        layout.addWidget(self.site_label)
        layout.addWidget(self.site_combo)

        self.setLayout(layout)

    def update_image(self):
        plate = self.plate_combo.currentText()
        well = self.well_combo.currentText()
        site = self.site_combo.currentText()
        print(plate, well, site)  # Debugging print

        # Select data based on plate, well, and site
        if (plate, well, site) in index_map:
            data_slice = index_map[(plate, well, site)]
            print(f"Updating image with data slice: {data_slice}")  # Debugging print
            
            # Update the image layer data correctly
            image_layer[0].data = final_dask_array[data_slice].compute()  # Ensure we compute the Dask array to numpy

# Add the navigation widget to the viewer
navigation_widget = NavigationWidget(plates, wells, sites)
viewer.window.add_dock_widget(navigation_widget)

napari.run()


In [ ]:
napari.__version__

In [ ]:
# Dummy final_dask_array from earlier
#final_dask_array = da.random.random((1, 6, 1, 3, 4, 1843, 1843))

# Initialize Napari viewer
viewer = napari.Viewer()

# Add Dask array to the viewer with initial data. Note that channel axis is relative to the sliced part
image_layer = viewer.add_image(final_dask_array[0, 0, 0, :, :, :, :], channel_axis=1)

# Create pull-down for plates and wells
@magicgui(plate={"choices": plates}, well={"choices": wells}, site={"max": len(sites)-1, "label": "Site"}, call_button=False)
def navigation_widget(plate: str, well: str, site: int):
    print(plate, well, site)
    # Select data based on plate, well, and site
    if (plate, well, site) in index_map:
        data_slice = index_map[(plate, well, site)]
        image_layer.data = final_dask_array[data_slice]

# Add widget to Napari viewer
viewer.window.add_dock_widget(navigation_widget)
napari.run()

In [ ]:
from napari.utils import notifications
from qtpy.QtWidgets import QComboBox, QSlider, QLabel, QVBoxLayout, QWidget

# Napari viewer setup
viewer = napari.Viewer()

# Creating widgets
class CustomWidget(QWidget):
    def __init__(self, viewer):
        super().__init__()
        self.viewer = viewer
        
        # Plate dropdown
        self.plate_label = QLabel("Select Plate:")
        self.plate_dropdown = QComboBox()
        self.plate_dropdown.addItems(df['Plate'].unique())  # Populate with unique plates
        
        # Well dropdown
        self.well_label = QLabel("Select Well:")
        self.well_dropdown = QComboBox()
        
        # Site slider
        self.site_label = QLabel("Select Site:")
        self.site_slider = QSlider()
        self.site_slider.setMinimum(0)
        self.site_slider.setMaximum(df['Site'].nunique() - 1)
        
        # Layout
        layout = QVBoxLayout()
        layout.addWidget(self.plate_label)
        layout.addWidget(self.plate_dropdown)
        layout.addWidget(self.well_label)
        layout.addWidget(self.well_dropdown)
        layout.addWidget(self.site_label)
        layout.addWidget(self.site_slider)
        self.setLayout(layout)
        
        # Connect signals
        self.plate_dropdown.currentTextChanged.connect(self.update_wells)
        self.well_dropdown.currentTextChanged.connect(self.update_viewer)
        self.site_slider.valueChanged.connect(self.update_viewer)

    def update_wells(self):
        plate = self.plate_dropdown.currentText()
        wells = df[df['Plate'] == plate]['Well'].unique()
        self.well_dropdown.clear()
        self.well_dropdown.addItems(wells)
        
    def update_viewer(self):
        plate = self.plate_dropdown.currentText()
        well = self.well_dropdown.currentText()
        site = self.site_slider.value()

        # Fetch the corresponding data
        key = (plate, well, site)
        if key in index_mapping:
            # Display the Dask array slice corresponding to the selected plate, well, and site
            zsteps = index_mapping[key]['zstep']
            channels = index_mapping[key]['channels']
            notifications.show_info(f"Plate: {plate}, Well: {well}, Site: {site}, ZSteps: {zsteps}, Channels: {channels}")
            # Update viewer with the new image data for the selected well, site, etc.
            # Assuming the data is already loaded into dask_array, show the corresponding slice
            self.viewer.layers.clear()
            self.viewer.add_image(dask_array[plate_index, well_index, site_index])

widget = CustomWidget(viewer)
viewer.window.add_dock_widget(widget, area='right')

napari.run()

In [ ]:
import napari
from magicgui import magicgui
import dask.array as da

# Assuming `data` is your 6x1x3x4x1843x1843 Dask array
data_shape = (6, 1, 3, 4, 1843, 1843)  # Update this to the actual Dask array
dask_array = da.random.random(data_shape)  # Replace with actual dask array
dask_array = plate_stack

# Create Napari viewer
viewer = napari.Viewer()

# Add the dask array as an image layer, with the channels as separate dimensions
image_layer = viewer.add_image(
    dask_array, 
    channel_axis=3,  # Channel is the 4th dimension (zero-indexed)
    name='Plate Data'
)

# Create widgets to navigate through wells, sites, and ZSteps
@magicgui(
    well={'widget_type': 'Slider', 'min': 0, 'max': data_shape[0] - 1, 'label': 'Well'},
    site={'widget_type': 'Slider', 'min': 0, 'max': data_shape[1] - 1, 'label': 'Site'},
    zstep={'widget_type': 'Slider', 'min': 0, 'max': data_shape[2] - 1, 'label': 'ZStep'}
)
def update_data(well: int = 0, site: int = 0, zstep: int = 0):
    # Update the current slice based on selected well, site, and zstep
    viewer.dims.set_point(0, well)
    viewer.dims.set_point(1, site)
    viewer.dims.set_point(2, zstep)

# Add widget to Napari
viewer.window.add_dock_widget(update_data)

# Automatically adjust to display different channels (added as separate sliders)
viewer.dims.axis_labels = ['Wells', 'Sites', 'ZSteps', 'Channels', 'X', 'Y']

# Start Napari
napari.run()


In [ ]:
from qtpy.QtWidgets import QLabel, QWidget, QVBoxLayout

viewer = napari.view_image(
        A_out,
        channel_axis=2,
        name=wavelengths,
        colormap=colormap,
        #contrast_limits=[[200, 4095], [500, 4095], [200, 4095]],
        )

# Create a custom widget to display the stack index
class StackIndexWidget(QWidget):
    def __init__(self):
        super().__init__()
        self.label = QLabel("Current stack index: 0")
        #self.label_file = QLabel("Filename w1")
        self.label_plate = QLabel("Plate: N/A")
        self.label_well = QLabel("Well: N/A")
        self.label_site = QLabel("Site: N/A")
        layout = QVBoxLayout()
        layout.addWidget(self.label)
        #layout.addWidget(self.label_file)
        layout.addWidget(self.label_plate)
        layout.addWidget(self.label_well)
        layout.addWidget(self.label_site)
        self.setLayout(layout)

    def update_index(self, index):
        self.label.setText(f"Current stack index: {index}")
        #self.label_file.setText(f"Filename w1: {df.loc[index, 'FileName_w1']}")
        self.label_plate.setText(f"Plate: {df.loc[index, PLATE]}")
        self.label_well.setText(f"Well: {df.loc[index, WELL]}")
        self.label_site.setText(f"Site: {df.loc[index, SITE]}")

# Instantiate the custom widget
stack_index_widget = StackIndexWidget()

# Add the widget to Napari as a dock widget
viewer.window.add_dock_widget(stack_index_widget, name="Stack Index")

# Callback function to update the widget with the current stack index
def on_index_change(event):
    current_index = viewer.dims.current_step[0]
    stack_index_widget.update_index(current_index)

# Connect the callback to the dims event
viewer.dims.events.current_step.connect(on_index_change)


In [ ]:

# Group by ZStep to handle stacking of channels for each Z-slice
grouped_zstep = df3d.groupby(['Plate', 'Well', 'Site', 'ZStep'])

# Prepare to collect Dask arrays for each ZStep
dask_arrays = []

for (plate, well, site, zstep), zstep_group in grouped_zstep:

    channels = []
    for channel_path in zstep_group[PATH]:
        print(channel_path)
        img = AICSImage(channel_path)
        # Use img.get_image_dask_data() for lazy loading of data
        channels.append(img.get_image_dask_data())

    # Stack the images along a new dimension (channels)
    dask_array = da.stack(channels, axis=0)  # Stack along the first axis for channels
    dask_arrays.append((plate, well, site, zstep, dask_array))

# Convert to DataFrame for ZStep level
zstep_df = pd.DataFrame(dask_arrays, columns=['Plate', 'Well', 'Site', 'ZStep', 'DaskArray'])

# Now group by Site to combine Dask arrays
grouped_site = zstep_df.groupby(['Plate', 'Well', 'Site'])

final_dask_arrays = []

for (plate, well, site), group in grouped_site:
    site_arrays = [entry[4] for entry in group.itertuples(index=False)]  # Extract Dask arrays for stacking
    final_dask_array = da.concatenate(site_arrays, axis=0)  # Concatenate along the channel axis
    final_dask_arrays.append((plate, well, site, final_dask_array))

# Convert to DataFrame for Site level
site_df = pd.DataFrame(final_dask_arrays, columns=['Plate', 'Well', 'Site', 'DaskArray'])

# Finally, group by Well to combine arrays across Sites
grouped_well = site_df.groupby(['Plate', 'Well'])

well_dask_arrays = []

for (plate, well), group in grouped_well:
    well_arrays = [entry[3] for entry in group.itertuples(index=False)]  # Extract Dask arrays for stacking
    well_dask_array = da.concatenate(well_arrays, axis=0)  # Concatenate across sites
    well_dask_arrays.append((plate, well, well_dask_array))

# Convert to DataFrame for Well level
well_df = pd.DataFrame(well_dask_arrays, columns=['Plate', 'Well', 'DaskArray'])

# Check the final DataFrame with Dask arrays for each Well
print(well_df)


In [ ]:
import dask.array as da
import numpy as np

def concat_da2d(selection, filename_cols, directory):
    da_out = None
    sites = []
    for index, row in selection.iterrows():
        channels = []
        for f in filename_cols:
            path = directory / row[f]
            #print(str(path))
            img = AICSImage(path)
            #channels.append(img.data)
            channels.append(img.get_image_dask_data())
            #print(img.get_image_dask_data().shape)
        site = da.concatenate(channels, axis=2)
        sites.append(site)

    da_out = da.concatenate(sites, axis=0)
    return da_out



In [ ]:
import dask.array as da

dask_arrays = {}
index_map = {}

def channels2da(directory, filenames):
    return da.stack([AICSImage(directory / f).get_image_dask_data() for f in filenames])
    
for (plate, well, site), group in grouped3d.iterrows():
    # Assuming the file paths for the Z-slices are in group['file_paths']
    dask_array = da.stack([channels2da(orig, fileset) for fileset in group[filename_cols]])
    
    # Store the Dask array and its corresponding index
    dask_arrays[(well, site)] = dask_array
    index_map[(well, site)] = len(dask_arrays) - 1  # Map (well, site) to Dask array index


In [ ]:
A_out = concat_da2d(df2d, filename_cols, orig)
print(A_out.shape)